In [22]:
## Import modules and methods from PySpark library and create a session
import builtins
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,collect_set,sum,avg,min,max,countDistinct,count,desc,asc
from pyspark.sql.types import IntegerType, FloatType
spark = SparkSession.builder.appName("datascience_test").getOrCreate()

In [3]:
## Load dataframes from csv files and show their datatypes as a tree

df_city = spark.read.csv("content/city.csv", header=True, inferSchema=True )
#df_city.show()
df_city.printSchema()

df_country = spark.read.csv("content/country.csv", header=True, inferSchema=True )
#df_country.show()
df_country.printSchema()

df_countrylg = spark.read.csv("content/countrylanguage.csv", header=True, inferSchema=True )
#df_countrylg.show()
df_countrylg.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- CountryCode: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Population: integer (nullable = true)

root
 |-- Code: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Continent: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- SurfaceArea: double (nullable = true)
 |-- IndepYear: string (nullable = true)
 |-- Population: integer (nullable = true)
 |-- LifeExpectancy: string (nullable = true)
 |-- GNP: double (nullable = true)
 |-- GNPOld: string (nullable = true)
 |-- LocalName: string (nullable = true)
 |-- GovernmentForm: string (nullable = true)
 |-- HeadOfState: string (nullable = true)
 |-- Capital: string (nullable = true)
 |-- Code2: string (nullable = true)

root
 |-- CountryCode: string (nullable = true)
 |-- Language: string (nullable = true)
 |-- IsOfficial: string (nullable = true)
 |-- Percentage: double (nullable = true)



SQL QUERIES

In [4]:
## 1- What is the highest life expectancy?
print("question 1")
df_country_new = df_country.withColumn("LifeExpectancy", df_country["LifeExpectancy"].cast(FloatType())) #convert datatype into float
df_country_new.dtypes  #print all datatypes of that dataset

df_country_new.fillna(0).select("LifeExpectancy") #convert Null values into 0
df_country_new.select(max("LifeExpectancy")).show()


## 2- In how many states is life expectancy higher than 50 years?
print("question 2")
df_country2 = df_country_new.filter(df_country_new.LifeExpectancy > 50.0)
df_country2.select(count("LifeExpectancy")).show()


## 3- What is the average gross domestic product of all states?
print("question 3")
df_country.select(avg("GNP")).show()


## 4- What is the maximum gross domestic product?
print("question 4")
df_country.select(max("GNP")).show()


## 5- What is the minimum gross domestic product?
print("question 5")
df_country.select(min("GNP")).show()


## 6- What is the gross domestic product of the planet?
print("question 6")
df_country.select(sum("GNP")).show()


## 7- What is the total area of ​​all the states of the world?
print("question 7")
df_country.select(sum('SurfaceArea')).collect()  #with collect() it doesn't return a dataframe but only the data in an array
df_country.select(sum('SurfaceArea')).collect()[0][0] #in this way I can take raw data

question 1
+-------------------+
|max(LifeExpectancy)|
+-------------------+
|               83.5|
+-------------------+

question 2
+---------------------+
|count(LifeExpectancy)|
+---------------------+
|                  194|
+---------------------+

question 3
+------------------+
|          avg(GNP)|
+------------------+
|122823.88242677823|
+------------------+

question 4
+---------+
| max(GNP)|
+---------+
|8510700.0|
+---------+

question 5
+--------+
|min(GNP)|
+--------+
|     0.0|
+--------+

question 6
+------------+
|    sum(GNP)|
+------------+
|2.93549079E7|
+------------+

question 7


148956306.9

In [5]:
## 8- What is the total population of all states in the world?
print("question 8")
df_country.select(sum('Population')).show()


## 9- What is the per capita income (gross domestic product / number of inhabitants)
print("question 9")
df_country_withPci = df_country.withColumn("PCI", (col("GNP") / col("Population"))*1000000) #create a new column with those conditions
df_country_withPci.select("PCI").show()


## 10- What is the lowest per capita income (gross domestic product / number of inhabitants), 
#      excluding countries with no gross domestic product? 
print("question 10")
df_country10 = df_country_withPci.filter(col("GNP") != 0) 
df_country10.select(min("PCI")).show()


## 11- How much is the number of inhabitants of the most populous city and the least populous one?
print("question 11")
df_city.select(max("Population")).show()
df_city.select(min("Population")).show()


## 12- How many inhabitants does the most populous and least populous city in Western Europe have?
print("question 12")
df_city_country = df_city.join(df_country, df_city.CountryCode == df_country.Code).select(
  "*",
  df_city.Name.alias('CityName'),
  df_country.Name.alias('CountryName'),
  df_city.Population.alias('CityPopulation'),
  df_country.Population.alias('CountryPopulation')
).drop('Name', 'Population')
#df_city_country.printSchema()

my_res12 = df_city_country.filter(df_country.Region=="Western Europe") #filter only data that has that region
my_res12.select(max('CityPopulation')).show() #take cities with more
my_res12.select(min('CityPopulation')).show() #or less population


## 13- How many African cities are contained in the database?
print("question 13")
my_res13 = df_city_country.filter(df_country.Continent=="Africa")
my_res13.select(count("Continent")).show()


## 14- How many Italian cities are contained in the database?
print("question 14")
my_res14 = df_city.filter(df_city.CountryCode=="ITA")
my_res14.select(count("CountryCode")).show()

question 8
+---------------+
|sum(Population)|
+---------------+
|     6078749450|
+---------------+

question 9
+------------------+
|               PCI|
+------------------+
|8038.8349514563115|
|263.02816901408454|
| 516.2292281410157|
| 7900.000000000001|
| 942.3144772433259|
|  20897.4358974359|
| 8944.700460829492|
| 15553.46169602622|
| 9187.675523871247|
| 515.0568181818181|
| 4911.764705882353|
|              null|
|              null|
|            9000.0|
|18594.832150799535|
| 26182.06085172644|
| 533.6177915696923|
|134.87677371172518|
|24387.537845492727|
| 386.5835656880433|
+------------------+
only showing top 20 rows

question 10
+------------------+
|          min(PCI)|
+------------------+
|40.816326530612244|
+------------------+

question 11
+---------------+
|max(Population)|
+---------------+
|       10500000|
+---------------+

+---------------+
|min(Population)|
+---------------+
|             42|
+---------------+

question 12
+-------------------+
|max(CityPo

In [6]:
## 15- In how many cities is Italian spoken, assuming that a language is spoken in a city if that 
#      language is spoken in the state that contains it?
print("question 15")
df_city_countrylg = df_city.join(df_countrylg, df_city.CountryCode == df_countrylg.CountryCode).select(
  "*",
  df_city.Name.alias('CityName'),
  df_city.Population.alias('CityPopulation')
).drop('Name', 'Population')
# df_city_countrylg.printschema()

my_res15 = df_city_countrylg.filter(df_countrylg.Language=='Italian')
my_res15.select(count("CityName")).show() #count the result whatever column it is, it's the same cuz I used the filter


## 16- In how many countries is English spoken?
print("question 16")
my_res16 = df_countrylg.filter("Language == 'English'")
my_res16.select(count("Language")).show()


## 17- In how many countries is English the official language?
print("question 17")
my_res17 = df_countrylg.filter(
  (df_countrylg.Language  == "English") & (df_countrylg.IsOfficial  == "T"))
my_res17.select(count("Language")).show()


## 18- What is the average population of cities where German is spoken, always assuming that a language 
#      is spoken in a city if that language is spoken in the state that contains it?
print("question 18")
my_res18 = df_city_countrylg.filter(col("Language") == 'German')
print(my_res18)
my_res18.select(avg("CityPopulation")).show()


## 19- What is the average population of German cities?
print("question 19")
my_res19 = df_city_country.filter(col("CountryName") == 'Germany')
my_res19.select(avg('CityPopulation')).show()


## 20- What is the average of the percentages of official languages ​​spoken in Africa?
print("question 20")
df_country_countrylg = df_country.join(df_countrylg, df_country.Code == df_countrylg.CountryCode).select(
  "*",
  df_country.Name.alias('CountryName'),
  df_country.Population.alias('CountryPopulation')
).drop('Name', 'Population')
# df_country_countrylg.printschema()

my_res20 = df_country_countrylg.filter(
  (df_country.Continent == 'Africa') & (df_countrylg.IsOfficial == 'T'))
my_res20.select(avg(df_countrylg.Percentage)).show()


## 21- And of the European ones?
print("question 21")
my_res21 = df_country_countrylg.filter(
  (df_country.Continent == 'Europe') & (df_countrylg.IsOfficial == 'T'))
my_res21.select(avg(df_countrylg.Percentage)).show()

question 15
+---------------+
|count(CityName)|
+---------------+
|            857|
+---------------+

question 16
+---------------+
|count(Language)|
+---------------+
|             60|
+---------------+

question 17
+---------------+
|count(Language)|
+---------------+
|             44|
+---------------+

question 18
DataFrame[ID: int, CountryCode: string, District: string, CountryCode: string, Language: string, IsOfficial: string, Percentage: double, CityName: string, CityPopulation: int]
+-------------------+
|avg(CityPopulation)|
+-------------------+
| 302209.21016949153|
+-------------------+

question 19
+-------------------+
|avg(CityPopulation)|
+-------------------+
|  282209.4946236559|
+-------------------+

question 20
+-----------------+
|  avg(Percentage)|
+-----------------+
|40.35263157894737|
+-----------------+

question 21
+-----------------+
|  avg(Percentage)|
+-----------------+
|64.18983050847457|
+-----------------+



GROUP QUERIES

In [63]:
## 1- Make a list of all continents with the number of states belonging to each one ordered from 
#     the continent with the most to the one with the fewest states.
print("question 1")
my_group1 = df_country.groupby('Continent').count()
my_res1 = my_group1.sort(col("count").desc()).show()


## 2- Make a list of all forms of government with the number of states belonging to each, sorted 
#     from most popular to least popular form of government.
print("question 2")
my_group2_1 = df_country.groupBy().sum().show() #this one returns sum of all columns (of num type)
my_group2_2 = df_country.groupby('GovernmentForm').count()
my_res2 = my_group2_2.sort(col("count").desc()).show()


## 3- Make a list of all continents with the number of countries for each continent where gross 
#     domestic product has increased over the past year.
print("question 3")
my_comparison = df_country.filter(df_country.GNPOld < df_country.GNP)
my_group3 = my_comparison.groupby('Continent').count()
my_res3 = my_group3.sort(col("count").desc()).show()


## 4- Which continent has the most states and how many are there?
print("question 4")
my_group4_1 = df_country.groupby('Continent').max()
my_group4_1print = df_country.groupby('Continent').max().show()
my_group4_2 = df_country.groupby('Continent').count()
my_group4_2print = df_country.groupby('Continent').count().show()

my_res4 = my_group4_2.sort(col("count").desc()).first()
print(my_res4)


## 5- Make a list of all the countries with the number of cities contained in each of them, sorted by number of cities.
print("question 5")
my_group5 = df_city_country.groupby('CountryName').count()
my_res5 = my_group5.sort(col("count").desc()).show()


## 6- Make a list of all continents sorted by total area.
print("question 6")
my_group6 = df_country.groupby('Continent').sum("SurfaceArea")
my_res6 = my_group6.sort(col("sum(SurfaceArea)").desc()).show()


## 7- Make a list of all official languages ​​with the number of countries in which they are spoken, sorted by the latter.
print("question 7")
my_group7 = df_country_countrylg.filter(df_countrylg.IsOfficial == 'T').groupby('Language').count()
my_res7 = my_group7.sort(col("count").desc()).show()


## 7Bonus- Make a list of the languages ​​of the world, sorted by the number of countries in which they are spoken.
print("question 7Bonus")
my_group7_1 = df_country_countrylg.groupby('Language').count()
my_res7_1 = my_group7_1.sort(col("count").desc()).show()


question 1
+-------------+-----+
|    Continent|count|
+-------------+-----+
|       Africa|   58|
|         Asia|   51|
|       Europe|   46|
|North America|   37|
|      Oceania|   28|
|South America|   14|
|   Antarctica|    5|
+-------------+-----+

question 2
+----------------+---------------+------------+
|sum(SurfaceArea)|sum(Population)|    sum(GNP)|
+----------------+---------------+------------+
|   1.489563069E8|     6078749450|2.93549079E7|
+----------------+---------------+------------+

+--------------------+-----+
|      GovernmentForm|count|
+--------------------+-----+
|            Republic|  122|
|Constitutional Mo...|   29|
|    Federal Republic|   15|
|Dependent Territo...|   12|
|            Monarchy|    5|
|Territory of Aust...|    4|
|Constitutional Mo...|    4|
|Nonmetropolitan T...|    4|
|Overseas Departme...|    4|
|Socialistic Republic|    3|
|        US Territory|    3|
|Nonmetropolitan T...|    3|
|Commonwealth of t...|    2|
|Territorial Colle...|    2|
|

In [64]:
## 8- Make a list of the countries in which more than 8 languages ​​are spoken, sorted by the 
#     number of languages ​​spoken.
print("question 8")
my_group8 = df_country_countrylg.groupby('CountryName').agg({'Language':'count'})
my_group8_filt = my_group8.filter(col('count(Language)') > 8)
my_res8 = my_group8_filt.sort(col("count(Language)").desc()).show()


## 9- Make a list of all continents with the average life expectancy for each continent, sorted by 
#     average life expectancy.
print("question 9")
my_group9 = df_country.groupby('Continent').agg({'LifeExpectancy':'avg'})
my_res9 = my_group9.sort(col("avg(LifeExpectancy)").desc()).show()


## 10- Make a list of all the countries with the number of cities and the average population of the cities.
print("question 10")
my_group10 = df_city_country.groupby("CountryName").agg(
  avg('CityPopulation'), 
  count('CityName'))
my_res10 = my_group10.sort(col("CountryName").asc()).show()


## 11- Make a list of the states whose population is less than that of the most populous city in the world.
print("question 11")
mostPopCity = df_city.select(max("Population")).collect()[0][0]
my_res11 = df_country.select('Name', 'Population')\
  .filter(col('Population') < mostPopCity)\
  .sort(col('Population'), ascending=False).show()


## 12- Make a list of all forms of government with the number of states belonging to each and the 
#      average gross domestic product (excluding states that have zero), sorted by average gross domestic product.
print("question 12")
my_group12 = df_country.filter(col('GNP') != 0.0).groupby("GovernmentForm").agg(
  count('Code'), 
  avg('GNP'))
my_res12 = my_group12.sort(col("avg(GNP)").desc()).show()


## 13- Make a list of the number of cities contained on each continent.
print("question 13")
my_sel13 = df_city_country.select('Continent', 'CityName')
my_group13 = my_sel13.groupby("Continent").agg(count('CityName'))
my_res13 = my_group13.sort(col("count(CityName)").desc()).show()


## 14- Make a list of the ratio of city population (the sum of city populations) to total state 
#      population for each state in the world and sort the list from highest to lowest ratio.
print("question 14")
df_cities_population = df_city.groupby("CountryCode").agg(sum('Population').alias('CitiesPopulation'))
df_country_cities_pop = df_country.join(df_cities_population, df_country.Code == df_cities_population.CountryCode).select(
  df_country.Name.alias('CountryName'),
  df_country.Population.alias('CountryPopulation'),
  df_cities_population.CitiesPopulation)

my_res14 = df_country_cities_pop\
  .select(col('CountryName'), (col('CitiesPopulation') / col('CountryPopulation')).alias('RatioPopulationCities'))\
  .sort(col("RatioPopulationCities").desc()).show()



question 8
+--------------------+---------------+
|         CountryName|count(Language)|
+--------------------+---------------+
|              Canada|             12|
|       United States|             12|
|               China|             12|
|  Russian Federation|             12|
|               India|             12|
|        South Africa|             11|
|            Tanzania|             11|
|                Iran|             10|
|Congo, The Democr...|             10|
|              Uganda|             10|
|          Mozambique|             10|
|               Sudan|             10|
|         Philippines|             10|
|             Nigeria|             10|
|               Kenya|             10|
|              Angola|              9|
|           Indonesia|              9|
|             Vietnam|              9|
+--------------------+---------------+

question 9
+-------------+-------------------+
|    Continent|avg(LifeExpectancy)|
+-------------+-------------------+
|       Eur

SUBQUERIES

In [9]:
## 1- Which country has the largest gross domestic product?
print("question 1")
my_sel1 = df_country.select('Name', 'GNP')
max_gnp = my_sel1.select(max('GNP')).collect()[0][0]
sq1 = my_sel1.filter(my_sel1.GNP == max_gnp).show()


## 2- Which country has the largest number of inhabitants?
print("question 2")
my_sel2 = df_country.select('Name', 'Population')
max_pop = my_sel2.select(max('Population')).collect()[0][0]
sq2 = my_sel2.filter(my_sel2.Population == max_pop).show()


## 3- Which country has the highest per capita income?
print("question 3")
my_sel3 = df_country_withPci.select('Name', 'PCI')
max_pci = my_sel3.select(max('PCI')).collect()[0][0]
sq3 = my_sel3.filter(my_sel3.PCI == max_pci).show()


## 4- What is the country with the lowest per capita income, excluding the countries that have zero GNP?
print("question 4")
my_sel4 = df_country_withPci.filter(col("GNP") != 0).select('Name', 'PCI')
min_pci = my_sel4.select(min('PCI')).collect()[0][0]
sq4 = my_sel4.filter(my_sel4.PCI == min_pci).show()


## 5- What is the oldest state?
print("question 5")
df_country_sq = df_country.withColumn("IndepYear", df_country["IndepYear"].cast(IntegerType()))
my_sel56 = df_country_sq.select('Name', 'IndepYear')
oldest_country = my_sel56.select(min('IndepYear')).collect()[0][0]
sq5 = my_sel56.filter(my_sel56.IndepYear == oldest_country).show()


## 6- And what is the most recent status?
print("question 6")
newest_country = my_sel56.select(max('IndepYear')).collect()[0][0]
sq6 = my_sel56.filter(my_sel56.IndepYear == newest_country).show()


## 7- What is the percentage of states in which life expectancy is greater than 50 years out of the total of states?
print("question 7")
tot_nations = df_country.count()
life_exp_50 = df_country.filter(col('LifeExpectancy') > 50.0).count() #with 50 instead 50.0 change the result cuz it's float
print(life_exp_50 / tot_nations, end="\n\n")


## 8- Which Asian country has the highest per capita income?
print("question 8")
my_sel8 = df_country_withPci.filter(col("Continent") == "Asia").select('Name', 'PCI')
max_pci_asia = my_sel8.select(max('PCI')).collect()[0][0]
sq8 = my_sel8.filter(my_sel8.PCI == max_pci_asia).show()


question 1
+-------------+---------+
|         Name|      GNP|
+-------------+---------+
|United States|8510700.0|
+-------------+---------+

question 2
+-----+----------+
| Name|Population|
+-----+----------+
|China|1277558000|
+-----+----------+

question 3
+----------+-----------------+
|      Name|              PCI|
+----------+-----------------+
|Luxembourg|37459.26095937572|
+----------+-----------------+

question 4
+--------------------+------------------+
|                Name|               PCI|
+--------------------+------------------+
|Sao Tome and Prin...|40.816326530612244|
+--------------------+------------------+

question 5
+-----+---------+
| Name|IndepYear|
+-----+---------+
|China|    -1523|
+-----+---------+

question 6
+-----+---------+
| Name|IndepYear|
+-----+---------+
|Palau|     1994|
+-----+---------+

question 7
0.8117154811715481

question 8
+------+----------------+
|  Name|             PCI|
+------+----------------+
|Brunei|35685.9756097561|
+------+----

In [73]:
## 9- Which is the African city with the largest number of inhabitants?
print("question 9")
my_sel9 = df_city_country.filter(col("Continent") == "Africa").select(
  'CityName',
  'CityPopulation')
max_pop_africa = my_sel9.select(max('CityPopulation')).collect()[0][0]
sq9 = my_sel9.filter(col('CityPopulation') == max_pop_africa).show()


## 13- What is the Middle East city whose population is closest to the average population of the Middle East cities?
print("question 13")
middleeast_cities = df_city_country\
  .filter(col("Region")=="Middle East")\
  .select("CityName", "CityPopulation")\
  .sort(col("CityPopulation").desc())
avg_pop_middleeast = middleeast_cities.select(avg("CityPopulation")).collect()[0][0]

cities_list = middleeast_cities.select('CityName').rdd.flatMap(lambda x: x).collect()
cities_pop_list = middleeast_cities.select('CityPopulation').rdd.flatMap(lambda x: x).collect()

value_closest = builtins.min(cities_pop_list, key=lambda x:abs(x-avg_pop_middleeast))
value_index_closest = builtins.min(range(len(cities_pop_list)), key=lambda i: abs(my_list[i]-avg_pop_middleeast))
print(value_closest)
print(cities_list[value_index_closest])
print('')


## 14- Make a list of the number of cities contained in each state and write the sum of the number 
#      of cities contained in the states that have more than 100.
print("question 14")
countries_tot_cities = df_city_country.groupby("CountryName")\
  .agg(count('CityName').alias('TotCities'))\
  .sort(col("TotCities").desc())

countries_more100_cities = countries_tot_cities\
  .sort(col("TotCities").desc())\
  .filter(col('TotCities') > 100)
countries_more100_cities.select(sum("TotCities")).show()


## 16- Assuming that only the official state language is spoken in cities, make a list of the 
#      languages ​​with the number of citizens who speak them from most to least spoken.
print("question 16")
languages_off_cities = df_city_countrylg.filter(col('IsOfficial') == 'T')\
  .groupby("Language").agg(sum('CityPopulation'))\
  .sort(col("sum(CityPopulation)").desc())
languages_off_cities.show()



question 9
+--------+--------------+
|CityName|CityPopulation|
+--------+--------------+
|   Cairo|       6789479|
+--------+--------------+

question 13
405905
Sanliurfa

question 14
+--------------+
|sum(TotCities)|
+--------------+
|          1974|
+--------------+

question 16
+----------+-------------------+
|  Language|sum(CityPopulation)|
+----------+-------------------+
|   Chinese|          179971347|
|   Spanish|          176842275|
|   English|          149478361|
|     Hindi|          123298526|
|Portuguese|           87843073|
|    Arabic|           80454565|
|  Japanese|           77965107|
|   Russian|           74703800|
|     Malay|           46130053|
|    Korean|           45476644|
|      Urdu|           31546745|
|    German|           31244367|
|  Pilipino|           30934791|
|   Turkish|           28676428|
|    French|           28153115|
|   Persian|           26032990|
| Ukrainian|           20074000|
|   Italian|           16008770|
|    Ketšua|           15